In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Dense, Dropout



In [2]:
data = pd.read_csv('content/final_data.csv')

In [3]:
import pandas as pd

# 假設 data_sorted 是你已經排序好的資料
data_sorted = data.copy()

# 獲取所有數值型特徵列，排除 'playerid' 和 'gameid'
columns_to_exclude = ['playerid', 'gameid', 'home', 'date', 'team', 'name', 'home_basic', 'win']
features_to_replace = data_sorted.select_dtypes(include=['float64', 'int64']).columns
features_to_replace = [col for col in features_to_replace if col not in columns_to_exclude]

# 定義前幾場比賽的數據平均值，這個變量可以調整
num_games_to_average = 4  # 可以調整這個值，表示取前幾場的數據平均

# 從最後一行開始，往前尋找對應的 playerid 的前幾場比賽
for i in range(len(data_sorted) - 1, -1, -1):
    current_player = data_sorted.loc[i, 'playerid']

    # 保存找到的前幾場比賽數據的列表
    previous_games = []

    # 遍歷查找前幾場比賽
    for j in range(i - 1, -1, -1):
        if data_sorted.loc[j, 'playerid'] == current_player and data_sorted.loc[j, 'SEC'] > 0:
            previous_games.append(data_sorted.loc[j, features_to_replace])
        if len(previous_games) >= num_games_to_average:
            break

    # 如果找到的前幾場比賽數據不為空，計算平均值
    if len(previous_games) > 0:
        average_values = pd.DataFrame(previous_games).mean()
        data_sorted.loc[i, features_to_replace] = average_values
    else:
        # 如果沒有找到前幾場比賽，將當前行數據設為 0
        data_sorted.loc[i, features_to_replace] = 0

# 查看結果
data_sorted.head()


C:\Users\barry\AppData\Local\Temp\ipykernel_35888\3992774499.py:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '39.25' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data_sorted.loc[i, features_to_replace] = average_values
C:\Users\barry\AppData\Local\Temp\ipykernel_35888\3992774499.py:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data_sorted.loc[i, features_to_replace] = average_values
C:\Users\barry\AppData\Local\Temp\ipykernel_35888\3992774499.py:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '5.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data_sorted

,gameid,playerid,estOFFRTG,OFFRTG,estDEFRTG,DEFRTG,estNETRTG,NETRTG,ASTpct,ASTtoTOV,...,passes,contestedFGM,contestedFGA,contestedFGpct,uncontestedFGM,uncontestedFGA,uncontestedFGpct,defendedatrimFGM,defendedatrimFGA,defendedatrimFGpct
0,22200001,1627759,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,22200001,1628369,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,22200001,201143,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,22200001,1628401,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,22200001,203935,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
grouped_data = pd.DataFrame(data_sorted)
grouped_data.to_csv('content/LookingFoward_data_avg_d3_4.csv', index=False)

: 

In [ ]:
import pandas as pd

# 假設 data_sorted 是你已經排序好的資料
data_sorted = data.copy()

# 獲取所有數值型特徵列，排除 'playerid'、'gameid' 和 'SEC'
columns_to_exclude = ['playerid', 'gameid', 'SEC']
features_to_replace = data_sorted.select_dtypes(include=['float64', 'int64']).columns
features_to_replace = [col for col in features_to_replace if col not in columns_to_exclude]

# 定義前幾場比賽的數據平均值，這個變量可以調整
num_games_to_average = 2  # 可以調整這個值，表示取前幾場的數據平均

# 使用 groupby 分組，避免逐行遍歷
grouped = data_sorted.groupby('playerid')

# 定義一個函數來處理每個分組
def process_group(group):
    # 遍歷每個 group，找出 SEC <= 0 的比賽，並用前幾場 SEC > 10 的數據進行填充
    for i in range(len(group) - 1, -1, -1):
        if group.iloc[i]['SEC'] <= 0:
            # 選取前幾場比賽 SEC > 10 的數據
            previous_games = group.iloc[:i][group.iloc[:i]['SEC'] > 10].tail(num_games_to_average)
            
            # 如果找到了符合條件的比賽數據，計算平均
            if len(previous_games) > 0:
                group.iloc[i, group.columns.get_loc(features_to_replace)] = previous_games[features_to_replace].mean().values
            else:
                # 如果沒有找到，將特徵值設為 0
                group.iloc[i, group.columns.get_loc(features_to_replace)] = 0
    
    return group

# 對每個 playerid 的分組應用該函數
data_sorted = grouped.apply(process_group)

# 查看結果
data_sorted.head()


In [ ]:
data = pd.read_csv('content/LookingFoward_data_avg_d3_2.csv')

In [7]:
data

,gameid,playerid,estOFFRTG,OFFRTG,estDEFRTG,DEFRTG,estNETRTG,NETRTG,ASTpct,ASTtoTOV,...,passes,contestedFGM,contestedFGA,contestedFGpct,uncontestedFGM,uncontestedFGA,uncontestedFGpct,defendedatrimFGM,defendedatrimFGA,defendedatrimFGpct
0,22200001,1627759,0.00,0.00,0.00,0.00,0.00,0.00,0.0000,0.00,...,0.0,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0,0.0,0.00
1,22200001,1628369,0.00,0.00,0.00,0.00,0.00,0.00,0.0000,0.00,...,0.0,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0,0.0,0.00
2,22200001,201143,0.00,0.00,0.00,0.00,0.00,0.00,0.0000,0.00,...,0.0,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0,0.0,0.00
3,22200001,1628401,0.00,0.00,0.00,0.00,0.00,0.00,0.0000,0.00,...,0.0,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0,0.0,0.00
4,22200001,203935,0.00,0.00,0.00,0.00,0.00,0.00,0.0000,0.00,...,0.0,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0,0.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39595,52200211,1630544,76.90,80.38,106.58,95.80,-29.66,-15.42,0.1040,0.48,...,22.8,1.4,2.0,0.2380,1.4,3.6,0.3332,0.8,1.2,0.50
39596,52200211,1630526,90.18,94.72,123.00,113.14,-32.82,-18.38,0.0348,0.40,...,17.0,0.6,1.8,0.2166,1.0,3.4,0.2244,2.4,3.0,0.65
39597,52200211,0,0.00,0.00,0.00,0.00,0.00,0.00,0.0000,0.00,...,0.0,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0,0.0,0.00
39598,52200211,0,0.00,0.00,0.00,0.00,0.00,0.00,0.0000,0.00,...,0.0,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0,0.0,0.00


In [58]:
# grouped_data

In [59]:
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
# 標準化
scaler = MinMaxScaler()

# 主客場
data['home_basic'] = data['home_basic'].apply(lambda x: 1 if x == 'TRUE' else 0)
# 假设 'win' 列包含 'TRUE' 和 'FALSE' 字符串
data['win'] = data['win'].map({'TRUE': 1, 'FALSE': 0})

# 如果你要对所有包含 'TRUE' 和 'FALSE' 的列进行转换
bool_columns = data.columns[data.isin(['TRUE', 'FALSE']).any()]

# 对所有这些列进行转换
for col in bool_columns:
    data[col] = data[col].map({'TRUE': 1, 'FALSE': 0})

In [60]:
# import pandas as pd

# def filter_games_by_players(data, player_ids):
#     # 找出這些 player 參與的所有比賽
#     player_games = data[data['playerid'].isin(player_ids)]['gameid'].unique()

#     # 初始化結果存儲
#     result = []

#     # 檢查每一場比賽，並確保不重複篩選其他已在同場比賽中的 player
#     for game in player_games:
#         # 確認這場比賽中的所有 player
#         players_in_game = data[data['gameid'] == game]['playerid'].unique()
#         result.append({
#             'gameid': game,
#             'players': list(players_in_game)
#         })

#     return result

# def get_filtered_game_data(data, player_ids):
#     # 使用上述 function 找出 players 參與的比賽
#     game_info = filter_games_by_players(data, player_ids)

#     # 找出這些比賽的所有資料
#     game_ids = [game['gameid'] for game in game_info]

#     # 根據 gameid 篩選出所有相關資料
#     filtered_data = data[data['gameid'].isin(game_ids)]

#     return filtered_data

# # 載入數據
# file_path = 'path_to_your_file.csv'  # 替換為實際檔案路徑
# # data = pd.read_csv(file_path)

# # 合併 player ids
# team1_player_ids = [1627759, 1629626, 201143, 1628401, 203935, 203518, 1626153, 1629602, 1629006, 201959, 1630242, 1627749, 1629019, 1626164, 202699]
# team2_player_ids = [202699, 200782, 203954, 1630178, 201935, 203507, 201566, 1629630, 202694, 1628389, 1628419, 1630162, 1628378, 201152, 201571]

# # 將兩個 list 合併
# combined_player_ids = team1_player_ids + team2_player_ids

# # 過濾出這些 player 的比賽資料
# filtered_game_data = get_filtered_game_data(data, combined_player_ids)

# # 顯示篩選後的資料
# print(filtered_game_data)

# filtered_game_dataCSV = pd.DataFrame(filtered_game_data)
# filtered_game_dataCSV.to_csv('content/filtered_game_data.csv', index=False)

In [61]:
file_path = 'content/selected_columns_s.txt'

with open(file_path, 'r') as file:
    selected_columns = [line.strip() for line in file]

print(selected_columns)

['plusminusPTS', 'estNETRTG', 'REBpct', 'estOFFRTG', 'DRB', 'FTM', 'uncontestedFGM', 'passes', 'ASTpct', 'touches', 'estUSGpct', 'oppPIP', 'TRB', 'oppFBPTS', 'PTS', 'estpace', 'FGpct', 'pie', 'DIST', 'defendedatrimFGA', 'BLK', '2ndPTS', 'TOVratio', 'pace', 'paceper40', '2ndAST', 'contestedFGA', 'OFFRTG', 'TSpct', 'FGM', 'playerid', 'team', '3Ppct', 'FTAST', 'STL', 'defendedatrimFGM', 'SEC', 'defendedatrimFGpct', 'oppPTSoffTOV', 'FBPTS', 'contestedFGM', '3PM', 'name', 'PIP', 'home', 'SPD', 'FTpct', 'PTSoffTOV', 'DEFRTG', 'contestedFGpct']


In [62]:
# 依照gameid進行分組，每一組代表一場比賽
games = data.groupby('gameid')

In [63]:
playerids = data.groupby('playerid')

In [64]:
# 將每場比賽中的所有球員數據合併成一個特徵向量
X = []
y = []

for gameid, group in games:
    if len(group) != 30:  # 每場比賽最多有30位球員
        continue

    # 將每場比賽中的所有球員數據合併成一個特徵向量
    #game_features = group.drop(columns=['gameid', 'win', 'home_basic','date']).values.flatten(
    game_features = group[selected_columns].values.flatten()
    game_features = np.append(game_features, group['date'].iloc[0])
    game_features = np.append(game_features, group['win'].iloc[0])
    X.append(game_features)
    #y.append(group['win'].iloc[0])

In [65]:
X = np.array(X)

X = pd.DataFrame(X)
#X.to_csv('/content/X.csv', index=False)
# 根據日期的值對所有行進行排序
#X_sorted = X.sort_values(by=X.columns[1501], ascending=True)
# 顯示排序後的DataFrame
#X_sorted.head()

In [66]:
X

,0,1,2,3,4,5,6,7,8,9,...,1492,1493,1494,1495,1496,1497,1498,1499,1500,1501
0,0.0,0.00,0.0000,0.00,0.0,0.0,0.0,0.0,0.0000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.00,0.0000,0.00,0.0,0.0,0.0,0.0,0.0000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.00,0.0000,0.00,0.0,0.0,0.0,0.0,0.0000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.00,0.0000,0.00,0.0,0.0,0.0,0.0,0.0000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.00,0.0000,0.00,0.0,0.0,0.0,0.0,0.0000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1315,1.2,1.14,0.0546,114.74,2.6,0.4,4.4,28.6,0.0992,43.6,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,749.6,0.0
1316,-5.8,-9.92,0.0450,103.72,1.0,0.4,0.4,5.4,0.0200,7.6,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,847.6,1.0
1317,4.0,4.02,0.0386,117.94,2.4,6.2,5.2,27.6,0.0626,45.4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,757.6,0.0
1318,-6.0,-8.80,0.0490,104.12,2.6,6.8,5.0,47.2,0.2968,73.6,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,844.0,1.0


In [67]:
df = X.reset_index(drop=True)

In [68]:
df.shape

(1320, 1502)

In [69]:
type(df)

pandas.core.frame.DataFrame

In [70]:
df

,0,1,2,3,4,5,6,7,8,9,...,1492,1493,1494,1495,1496,1497,1498,1499,1500,1501
0,0.0,0.00,0.0000,0.00,0.0,0.0,0.0,0.0,0.0000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.00,0.0000,0.00,0.0,0.0,0.0,0.0,0.0000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.00,0.0000,0.00,0.0,0.0,0.0,0.0,0.0000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.00,0.0000,0.00,0.0,0.0,0.0,0.0,0.0000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.00,0.0000,0.00,0.0,0.0,0.0,0.0,0.0000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1315,1.2,1.14,0.0546,114.74,2.6,0.4,4.4,28.6,0.0992,43.6,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,749.6,0.0
1316,-5.8,-9.92,0.0450,103.72,1.0,0.4,0.4,5.4,0.0200,7.6,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,847.6,1.0
1317,4.0,4.02,0.0386,117.94,2.4,6.2,5.2,27.6,0.0626,45.4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,757.6,0.0
1318,-6.0,-8.80,0.0490,104.12,2.6,6.8,5.0,47.2,0.2968,73.6,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,844.0,1.0


In [71]:
# 找到"team"的位置
team_index = [i for i, col in enumerate(selected_columns) if "team" in col]

# 輸出結果
print(team_index)

[31]


In [72]:
y = np.array(df[1501])

# 勝負
y = np.where(y == 1, 1, 0)
y = to_categorical(y, 2)

In [73]:
Y = pd.DataFrame(y)

In [74]:
Y

,0,1
0,0.0,1.0
1,0.0,1.0
2,0.0,1.0
3,1.0,0.0
4,0.0,1.0
...,...,...
1315,1.0,0.0
1316,0.0,1.0
1317,1.0,0.0
1318,0.0,1.0


In [75]:
df.drop(df.columns[1501], axis=1, inplace=True)

In [76]:
df

,0,1,2,3,4,5,6,7,8,9,...,1491,1492,1493,1494,1495,1496,1497,1498,1499,1500
0,0.0,0.00,0.0000,0.00,0.0,0.0,0.0,0.0,0.0000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.00,0.0000,0.00,0.0,0.0,0.0,0.0,0.0000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.00,0.0000,0.00,0.0,0.0,0.0,0.0,0.0000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.00,0.0000,0.00,0.0,0.0,0.0,0.0,0.0000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.00,0.0000,0.00,0.0,0.0,0.0,0.0,0.0000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1315,1.2,1.14,0.0546,114.74,2.6,0.4,4.4,28.6,0.0992,43.6,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,749.6
1316,-5.8,-9.92,0.0450,103.72,1.0,0.4,0.4,5.4,0.0200,7.6,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,847.6
1317,4.0,4.02,0.0386,117.94,2.4,6.2,5.2,27.6,0.0626,45.4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,757.6
1318,-6.0,-8.80,0.0490,104.12,2.6,6.8,5.0,47.2,0.2968,73.6,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,844.0


In [77]:
X_train, X_test, y_train, y_test = train_test_split(df, Y, test_size=0.3, shuffle = False)
X_train_df = pd.DataFrame(X_train)
# X_train_df.to_csv('content/X_trainA.csv', index=False)

In [78]:
X_train

,0,1,2,3,4,5,6,7,8,9,...,1491,1492,1493,1494,1495,1496,1497,1498,1499,1500
0,0.0,0.00,0.0000,0.00,0.0,0.0,0.0,0.0,0.0000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.00,0.0000,0.00,0.0,0.0,0.0,0.0,0.0000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.00,0.0000,0.00,0.0,0.0,0.0,0.0,0.0000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.00,0.0000,0.00,0.0,0.0,0.0,0.0,0.0000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.00,0.0000,0.00,0.0,0.0,0.0,0.0,0.0000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
919,12.0,17.58,0.0544,118.36,3.0,1.2,4.4,27.6,0.0560,39.2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,558.6
920,7.4,12.12,0.0538,118.96,3.2,3.8,2.2,22.6,0.1326,40.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,559.6
921,-8.2,-18.24,0.0858,105.86,2.4,0.4,1.0,28.4,0.1210,35.2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,558.8
922,-0.8,-4.48,0.0756,118.32,3.6,3.0,4.0,21.4,0.0950,36.2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,560.6


In [79]:
X_test

,0,1,2,3,4,5,6,7,8,9,...,1491,1492,1493,1494,1495,1496,1497,1498,1499,1500
924,-1.0,-0.04,0.0384,107.74,2.4,0.8,1.8,23.2,0.0756,32.4,...,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.00,0.0000,531.6
925,-18.0,-31.16,0.1026,94.60,5.0,4.2,1.8,33.0,0.1336,48.2,...,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.00,0.0000,561.2
926,0.8,-0.32,0.0464,102.88,1.8,1.2,3.0,20.0,0.0784,31.6,...,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.00,0.0000,557.6
927,5.0,9.46,0.0924,117.20,3.8,3.4,3.0,23.8,0.0890,41.0,...,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.00,0.0000,567.2
928,2.0,-4.22,0.0310,128.86,0.6,0.0,1.8,11.0,0.1878,15.0,...,0.2,96.0,6.8,12.8,3.952,0.4,0.8,117.26,0.5642,463.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1315,1.2,1.14,0.0546,114.74,2.6,0.4,4.4,28.6,0.0992,43.6,...,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.00,0.0000,749.6
1316,-5.8,-9.92,0.0450,103.72,1.0,0.4,0.4,5.4,0.0200,7.6,...,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.00,0.0000,847.6
1317,4.0,4.02,0.0386,117.94,2.4,6.2,5.2,27.6,0.0626,45.4,...,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.00,0.0000,757.6
1318,-6.0,-8.80,0.0490,104.12,2.6,6.8,5.0,47.2,0.2968,73.6,...,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.00,0.0000,844.0


In [80]:
y_train

,0,1
0,0.0,1.0
1,0.0,1.0
2,0.0,1.0
3,1.0,0.0
4,0.0,1.0
...,...,...
919,1.0,0.0
920,0.0,1.0
921,1.0,0.0
922,1.0,0.0


In [81]:
y_test

,0,1
924,0.0,1.0
925,1.0,0.0
926,0.0,1.0
927,1.0,0.0
928,1.0,0.0
...,...,...
1315,1.0,0.0
1316,0.0,1.0
1317,1.0,0.0
1318,0.0,1.0


In [82]:
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [83]:
print(y_train)

       0    1
0    0.0  1.0
1    0.0  1.0
2    0.0  1.0
3    1.0  0.0
4    0.0  1.0
..   ...  ...
919  1.0  0.0
920  0.0  1.0
921  1.0  0.0
922  1.0  0.0
923  1.0  0.0

[924 rows x 2 columns]


In [84]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

dropout_rate = 0.2

# Reshape the input data correctly
X_train = X_train.reshape(X_train.shape[0], 1, 1501)  # (samples, time steps, features)
X_test = X_test.reshape(X_test.shape[0], 1, 1501)    # (samples, time steps, features)

my_model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(80, activation=tf.nn.tanh, return_sequences=True, input_shape=(1, 1501)), # Correct input shape
    tf.keras.layers.Dropout(dropout_rate),
    tf.keras.layers.LSTM(60, activation=tf.nn.tanh, return_sequences=True),
    tf.keras.layers.Dropout(dropout_rate),
    tf.keras.layers.LSTM(40, activation=tf.nn.tanh),
    tf.keras.layers.Dropout(dropout_rate),
    tf.keras.layers.Dense(20, activation=tf.nn.sigmoid),
    tf.keras.layers.Dropout(dropout_rate),
    tf.keras.layers.Dense(2, activation=tf.nn.softmax)
])

my_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 训练模型
my_model.fit(X_train, y_train, epochs=30, batch_size=32)

# 评估模型
loss, accuracy = my_model.evaluate(X_test, y_test, verbose=0)

print(f"模型准确性: {accuracy:.2f}")

Epoch 1/30


c:\Users\barry\.conda\envs\nba_1\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


29/29 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - accuracy: 0.5692 - loss: 0.7090
Epoch 2/30
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5871 - loss: 0.6851
Epoch 3/30
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5988 - loss: 0.6766
Epoch 4/30
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5825 - loss: 0.6884
Epoch 5/30
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5370 - loss: 0.6992
Epoch 6/30
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6028 - loss: 0.6721
Epoch 7/30
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6154 - loss: 0.6626
Epoch 8/30
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5550 - loss: 0.6845
Epoch 9/30
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6202 - loss: 0.6685
Epoch 10/30
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6097 - loss: 0.6742
Epoch 11/30
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5818 - loss: 0.6853
Epoch 12/30
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6033 - loss: 0.6657


In [85]:
X_train

array([[[0.47916667, 0.4026397 , 0.        , ..., 0.        ,
         0.        , 0.        ]],

       [[0.47916667, 0.4026397 , 0.        , ..., 0.        ,
         0.        , 0.        ]],

       [[0.47916667, 0.4026397 , 0.        , ..., 0.        ,
         0.        , 0.        ]],

       ...,

       [[0.30833333, 0.24631471, 0.48529412, ..., 0.        ,
         0.        , 0.99678915]],

       [[0.4625    , 0.36424409, 0.42760181, ..., 0.        ,
         0.        , 1.        ]],

       [[0.40416667, 0.32773397, 0.31221719, ..., 0.        ,
         0.        , 1.        ]]])

In [86]:
y_train

,0,1
0,0.0,1.0
1,0.0,1.0
2,0.0,1.0
3,1.0,0.0
4,0.0,1.0
...,...,...
919,1.0,0.0
920,0.0,1.0
921,1.0,0.0
922,1.0,0.0


In [87]:
X_test

array([[[0.45833333, 0.40229688, 0.21719457, ..., 0.        ,
         0.        , 0.94826971]],

       [[0.10416667, 0.1355845 , 0.58031674, ..., 0.        ,
         0.        , 1.00107028]],

       [[0.49583333, 0.39989715, 0.26244344, ..., 0.        ,
         0.        , 0.99464859]],

       ...,

       [[0.5625    , 0.4370929 , 0.21832579, ..., 0.        ,
         0.        , 1.3514092 ]],

       [[0.35416667, 0.32721975, 0.27714932, ..., 0.        ,
         0.        , 1.50552979]],

       [[0.5125    , 0.44686322, 0.29072398, ..., 0.        ,
         0.        , 1.42454513]]])

In [88]:
y_test

,0,1
924,0.0,1.0
925,1.0,0.0
926,0.0,1.0
927,1.0,0.0
928,1.0,0.0
...,...,...
1315,1.0,0.0
1316,0.0,1.0
1317,1.0,0.0
1318,0.0,1.0


In [89]:
# 保存模型到指定的文件路徑
my_model.save('my_model.h5')

# 保存模型架構到 JSON 格式
model_json = my_model.to_json()
with open('my_model.json', 'w') as json_file:
    json_file.write(model_json)

In [90]:
# 函數：處理球隊 ID 並將結果存入 result_array
def process_player_ids(player_ids, data, selected_columns):
    for player_id in player_ids:
        found = False
        for idx in range(len(data) - 1, -1, -1):  # 從最後一行開始
            if data.loc[idx, 'playerid'] == player_id:  # 找到對應的 player_id
                # 提取該行的特徵數據
                selected_data = data.loc[idx, selected_columns]
                # 存到結果陣列中
                result_array.append(selected_data)
                found = True
                break  # 找到後跳出內部循環，進行下一個 player_id 的處理

        if not found:
            # 如果沒有找到該 player_id，填充一行全為 0 的數據
            zero_data = pd.Series([0] * len(selected_columns), index=selected_columns)
            result_array.append(zero_data)

In [91]:
import pandas as pd
import numpy as np

# 創建一個存儲結果的新陣列
result_array = []
# 增加球員ID到15位
team1_player_ids = [202699, 200782, 203954, 1630178, 201935, 203507, 201566, 1629630, 202694, 1628389, 1628419, 1630162, 1628378, 201152, 201571]
team2_player_ids = [1627759, 1629626, 201143, 1628401, 203935, 203518, 1626153, 1629602, 1629006, 201959, 1630242, 1627749, 1629019, 1626164, 202699]



# 處理 team1_player_ids
process_player_ids(team1_player_ids, data, selected_columns)

# 處理 team2_player_ids
process_player_ids(team2_player_ids, data, selected_columns)

# 將結果轉換為 DataFrame 方便查看
result_df = pd.DataFrame(result_array)



In [92]:
result_df

,plusminusPTS,estNETRTG,REBpct,estOFFRTG,DRB,FTM,uncontestedFGM,passes,ASTpct,touches,...,contestedFGM,3PM,name,PIP,home,SPD,FTpct,PTSoffTOV,DEFRTG,contestedFGpct
38565,-4.6,-8.12,0.1010,106.26,5.4,0.8,2.6,27.8,0.0630,39.2,...,1.6,0.8,500.0,5.2,15.0,4.078,0.4000,2.2,114.88,0.5334
38566,-5.0,-11.82,0.0616,103.94,2.4,0.2,1.8,18.0,0.0664,23.6,...,0.4,1.6,422.0,1.2,15.0,3.988,0.2000,0.6,116.50,0.4000
38567,-2.8,-5.50,0.1160,107.28,7.0,9.6,6.4,34.2,0.1070,63.8,...,2.2,0.8,259.0,9.2,15.0,3.520,0.9002,4.4,113.34,0.3866
38568,-3.6,-8.34,0.0730,105.40,4.6,2.2,5.0,47.2,0.1334,67.8,...,2.0,3.0,521.0,6.8,15.0,4.238,0.5666,2.6,113.22,0.3102
38569,-7.8,-9.54,0.0984,102.90,7.2,6.6,3.6,62.6,0.3748,87.8,...,2.2,1.8,227.0,6.4,15.0,3.396,0.8978,1.0,114.60,0.3842
37021,-0.2,-7.70,0.1650,115.86,7.6,4.6,3.0,36.4,0.2660,63.0,...,6.4,0.0,172.0,16.8,19.4,3.986,0.5586,3.2,123.88,0.6526
38179,-3.6,-8.06,0.0990,112.08,4.6,4.4,4.6,57.8,0.2998,86.6,...,5.0,2.2,457.0,13.6,19.6,3.820,0.6400,4.4,120.66,0.5072
37819,5.2,8.92,0.0926,112.08,5.4,4.0,5.0,46.0,0.3438,76.4,...,4.4,2.2,200.0,14.0,15.0,4.058,0.5158,5.8,104.94,0.4694
38176,-9.2,-16.62,0.0712,105.00,3.2,0.4,1.0,19.2,0.0692,29.0,...,1.4,0.6,363.0,2.0,13.0,3.950,0.2000,1.0,124.66,0.4134
39542,-5.8,-8.68,0.1344,102.64,8.6,4.2,4.2,54.2,0.1264,78.2,...,3.2,0.0,30.0,12.8,12.8,3.876,0.8600,2.6,111.28,0.3452


In [93]:
feature_vector = result_df.values.flatten()

feature_vector = np.append(feature_vector,776)
# 將一維向量轉換為 DataFrame 的一行
flattened_df = pd.DataFrame([feature_vector])

In [94]:
flattened_df

,0,1,2,3,4,5,6,7,8,9,...,1491,1492,1493,1494,1495,1496,1497,1498,1499,1500
0,-4.6,-8.12,0.101,106.26,5.4,0.8,2.6,27.8,0.063,39.2,...,0.8,500.0,5.2,15.0,4.078,0.4,2.2,114.88,0.5334,776.0


In [95]:
flattened_df = scaler.fit_transform(flattened_df)

In [96]:
input_vector = flattened_df.reshape(1, 1, 1501)
predicted_class = my_model.predict(input_vector)

# 打印預測的類別
print("Predicted class:", predicted_class)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 593ms/step
Predicted class: [[0.3093797  0.69062024]]


In [97]:
# 創建一個存儲結果的新陣列
result_array = []
# 增加球員ID到15位
team1_player_ids = [1627759, 1629626, 201143, 1628401, 203935, 203518, 1626153, 1629602, 1629006, 201959, 1630242, 1627749, 1629019, 1626164, 202699]
team2_player_ids = [202699, 200782, 203954, 1630178, 201935, 203507, 201566, 1629630, 202694, 1628389, 1628419, 1630162, 1628378, 201152, 201571]

# 處理 team1_player_ids
process_player_ids(team1_player_ids, data, selected_columns)

# 處理 team2_player_ids
process_player_ids(team2_player_ids, data, selected_columns)


In [98]:
result_df = pd.DataFrame(result_array)
result_df

,plusminusPTS,estNETRTG,REBpct,estOFFRTG,DRB,FTM,uncontestedFGM,passes,ASTpct,touches,...,contestedFGM,3PM,name,PIP,home,SPD,FTpct,PTSoffTOV,DEFRTG,contestedFGpct
39120,-4.0,-5.70,0.0714,106.16,2.8,2.2,3.6,34.8,0.1418,57.4,...,4.0,1.0,242.0,9.6,10.8,3.946,0.3400,1.6,114.66,0.5250
36564,-5.4,-23.56,0.1170,94.24,3.4,2.0,0.8,12.0,0.1310,21.4,...,1.2,0.2,40.0,2.8,15.0,4.168,0.5000,0.4,115.12,0.3700
39122,-1.8,-3.72,0.1194,108.56,4.6,0.2,1.8,29.8,0.1532,36.6,...,0.8,1.0,9.0,2.8,10.8,3.852,0.1000,1.6,112.46,0.6666
39123,4.8,12.50,0.0444,120.28,2.0,1.0,4.0,34.4,0.1010,46.8,...,0.8,3.6,126.0,2.4,10.8,4.016,0.3334,3.0,109.70,0.4666
39124,0.8,2.32,0.0726,112.28,3.6,2.8,3.2,37.6,0.1734,53.2,...,1.2,2.4,364.0,4.0,10.8,3.724,0.6766,3.8,109.68,0.3500
Unnamed 0,0.0,0.00,0.0000,0.00,0.0,0.0,0.0,0.0,0.0000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000,0.0000,0.0,0.00,0.0000
36640,4.2,4.54,0.1102,110.78,5.8,1.6,1.8,53.8,0.2416,66.8,...,2.2,1.0,117.0,5.6,27.6,4.292,0.5666,1.8,106.84,0.4908
Unnamed 1,0.0,0.00,0.0000,0.00,0.0,0.0,0.0,0.0,0.0000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000,0.0000,0.0,0.00,0.0000
38743,-8.2,-34.74,0.0340,92.22,0.2,0.2,0.2,7.2,0.1132,11.6,...,0.8,0.0,282.0,2.0,14.4,4.504,0.1000,0.0,126.10,0.6666
36639,-9.0,-40.36,0.0564,89.22,0.8,0.8,1.2,13.6,0.0988,18.2,...,0.4,0.8,485.0,0.8,28.0,4.100,0.4000,0.6,123.58,0.2000


In [99]:
feature_vector = result_df.values.flatten()
feature_vector = np.append(feature_vector,776)
# 將一維向量轉換為 DataFrame 的一行
flattened_df = pd.DataFrame([feature_vector])
flattened_df


,0,1,2,3,4,5,6,7,8,9,...,1491,1492,1493,1494,1495,1496,1497,1498,1499,1500
0,-4.0,-5.7,0.0714,106.16,2.8,2.2,3.6,34.8,0.1418,57.4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,776.0


In [100]:
#標準化
flattened_df = scaler.fit_transform(flattened_df)
input_vector = flattened_df.reshape(1, 1, 1501)
predicted_class = my_model.predict(input_vector)

# 打印預測的類別
print("Predicted class:", predicted_class)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Predicted class: [[0.3093797  0.69062024]]


data

In [101]:
games_data = pd.read_csv('/content/games.csv')


FileNotFoundError: [Errno 2] No such file or directory: '/content/games.csv'

In [ ]:
games_data

In [ ]:
final_data = pd.read_csv('content/final_data.csv')
final_data

In [ ]:
import pandas as pd

# 讀取資料
final_data = pd.read_csv('content/final_data.csv')
games_data = pd.read_csv('content/games.csv')

# 使用 merge 操作，基於 'gameid' 進行合併
merged_data = final_data.merge(games_data[['gameid', 'date']], on='gameid', how='left', suffixes=('', '_games'))

# 條件替換：當 playerid 不為 0 時，替換 final_data 中的 date
condition = final_data['playerid'] != 0
final_data.loc[condition, 'date'] = merged_data.loc[condition, 'date_games']

# 查看結果，確保數據被正確替換
print(final_data.head())

# 保存最終結果
final_data.to_csv('content/final_data_updated.csv', index=False)


In [ ]:
final_data.date

In [ ]:
import pandas as pd

# 假設 date 列已經存在於 final_data 中
# final_data = pd.read_csv('content/final_data.csv')

# 使用 pd.to_datetime 轉換日期格式，並提取日期部分
final_data['date'] = pd.to_datetime(final_data['date'], errors='coerce').dt.date

# 查看轉換結果
print(final_data['date'].head())

# 保存結果
final_data.to_csv('content/final_data_updated.csv', index=False)


In [ ]:
# 函數：處理球隊 ID 並將結果存入 result_array
def process_player_ids2(player_ids, data, selected_columns, date, result_array):
    # result_array = []  # 用來存放結果
    date = pd.to_datetime(date)  # 將輸入的日期轉換為 datetime 格式

    for player_id in player_ids:
        found_data = []  # 用來存放找到的符合條件的行
        found_count = 0  # 計算找到的符合條件的行數

        for idx in range(len(data) - 1, -1, -1):  # 從最後一行開始倒序遍歷
            # 找到符合條件的 player_id 且日期在所選日期之前
            if data.loc[idx, 'playerid'] == player_id and pd.to_datetime(data.loc[idx, 'date']) <= date:
                # 提取該行的特徵數據
                selected_data = data.loc[idx, selected_columns]
                # 將符合條件的數據添加到 found_data 列表中
                found_data.append(selected_data)
                found_count += 1

                # 如果已經找到了 5 行數據，則跳出循環
                if found_count == 5:
                    break

        # 如果找到的符合條件的行少於 5 行，則用 0 填充其餘的行
        while len(found_data) < 5:
            zero_data = pd.Series([0] * len(selected_columns), index=selected_columns)
            found_data.append(zero_data)

        # 將找到的數據存入結果陣列
        result_array.extend(found_data)

    return result_array

In [ ]:
import pandas as pd
import numpy as np

# 創建一個存儲結果的新陣列
result_arr1 = []
result_arr2 = []
# 增加球員ID到15位
team1_player_ids = [202699, 200782, 203954, 1630178, 201935, 203507, 201566, 1629630, 202694, 1628389, 1628419, 1630162, 1628378, 201152, 201571]
team2_player_ids = [1627759, 1629626, 201143, 1628401, 203935, 203518, 1626153, 1629602, 1629006, 201959, 1630242, 1627749, 1629019, 1626164, 202699]
date = '2023-06-20'


# 處理 team1_player_ids
process_player_ids2(team1_player_ids, final_data, selected_columns, date, result_arr1)

# 處理 team2_player_ids
process_player_ids2(team2_player_ids, final_data, selected_columns, date, result_arr2)

# 將結果轉換為 DataFrame 方便查看
result_df2 = pd.DataFrame(result_array2)



In [ ]:
result_df.shape

In [ ]:
result_df